In [16]:
#importando as bibliotecas que usarei: Pandas, Docx#
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [72]:
'''___________________________________________________________
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
_______________________________________'''


In [84]:
class BUSCA_DADOS:

    def __init__(self, valor):        
        banco_de_dados = pd.read_excel('0. Cadastro.xlsm', sheet_name= None)

        for nome_aba, dados_aba in banco_de_dados.items():            
            if valor in dados_aba.values:
                banco_de_dados = dados_aba
                break                 
        else:
            raise ValueError("O valor não foi encontrado nas abas do arquivo")

        self.linha = banco_de_dados.loc[banco_de_dados.isin([valor]).any(axis=1)].iloc[0] 

In [79]:
class PARTE:

    #busca no Banco de Dados o parâmetro 'parte' (qualquer valor = nome, CPF, etc)#
    def __init__(self, parte):
                      
        self.dicionario = BUSCA_DADOS(parte).linha.to_dict()   
        self._agrupar_endereco()
        self._qualificar_parte()
          
    def _agrupar_endereco(self):
                        
        self.dicionario['NUMERO_COMPLEMENTO'] = self.dicionario['NUMERO'] if pd.isna(self.dicionario['COMPLEMENTO'])\
            else f"{self.dicionario['NUMERO']} - {self.dicionario['COMPLEMENTO']}" 

        self.dicionario['ENDERECO'] = (f"{self.dicionario['LOGRADOURO']}, nº {self.dicionario['NUMERO_COMPLEMENTO']}, Bairro "
            f"{self.dicionario['BAIRRO']}, CEP {self.dicionario['CEP']}, {self.dicionario['CIDADE']}/{self.dicionario['UF']}")
   
    def _qualificar_parte(self):

        if 'CPF' in self.dicionario:

            self.dicionario['QUALIFICAÇÃO'] = '{}, {}, {}, {}, RG n° {} {}, CPF n° {}, e-mail: {}, com endereço à {}'.format(
                self.dicionario['NOME'], self.dicionario['NACIONALIDADE'], self.dicionario['ESTADO_CIVIL'], 
                self.dicionario['PROFISSAO'], self.dicionario['RG'], self.dicionario['EXPEDIDOR_RG'], 
                self.dicionario['CPF'], self.dicionario['EMAIL'],self.dicionario['ENDERECO'])

        elif self.dicionario['ESCRITORIO'] == 'GFA':
            self.dicionario['QUALIFICAÇÃO'] = (f"{self.dicionario['NOME']}, inscrito no CNPJ n° {self.dicionario['CNPJ']}, com "
                f"endereço à {self.dicionario['ENDERECO']}, devidamente representado neste ato pelo seu síndico(a) "
                f"{self.dicionario['ADMINISTRADOR']}")

        else:
            self.dicionario['QUALIFICAÇÃO'] = ('{}, pessoa jurídica inscrita no CNPJ n° {}, com endereço à {}, devidamente' 
                ' representada neste ato pelo(a) sócio(a) {}').format(self.dicionario['NOME'], self.dicionario['CNPJ'], 
                self.dicionario['ENDERECO'],self.dicionario['ADMINISTRADOR'])

In [87]:
class PECA:

    #cliente e adverso podem ser lista também (multipartes)#
    def __init__(self, peca, cliente, adverso = None):        
        
        self._carregar_atributos_e_valores(peca)
        self._criar_dicionario(cliente, adverso)
        self.criar_documento(peca)

    def _carregar_atributos_e_valores(self, peca):
        self.atributos = str(BUSCA_DADOS(peca).linha['ATRIBUTOS']).strip().upper().split('/')    
        self.valores = str(BUSCA_DADOS(peca).linha['VALORES']).strip().split('/')
       
    def _criar_dicionario(self, cliente, adverso):
        self.dicionario = dict(zip(self.atributos, self.valores))
        self._atualizar_dicionario_com_parte(cliente)
        if adverso:
            self._atualizar_dicionario_com_parte(adverso, 'ADVERSO_')  
    
    def _atualizar_dicionario_com_parte(self, parte, prefixo = 'CLIENTE_'):

        def dicionario_se_parte_unica(parte, prefixo):
            for chave, valor in PARTE(parte).dicionario.items():
                nova_chave = prefixo + chave
                self.dicionario[nova_chave] = valor
        
        #checa o número de parte se é multiplo(lista)#       
        if isinstance(parte, str):            
            dicionario_se_parte_unica(parte, prefixo)
            
        else:
            dicionario_se_parte_unica(parte[0], prefixo)
            self.dicionario['CLIENTE_QUALIFICAÇÃO'] = f"(1) {self.dicionario['CLIENTE_QUALIFICAÇÃO']}"

            for index, nome in enumerate(parte[1:], start=2):
                self.dicionario['CLIENTE_QUALIFICAÇÃO'] += f" e ({index}) {PARTE(nome).dicionario['QUALIFICAÇÃO']}"
   
    def criar_documento(self, peca):           
        documento = Document(f"{self.dicionario['CLIENTE_ESCRITORIO']}\{peca}.docx")
         
        chave_de_substituicao = {'x' + atributo + 'x': str(valor) for atributo, valor in self.dicionario.items()}
               
        for paragrafo in documento.paragraphs:
            for chave in chave_de_substituicao:
                if chave in paragrafo.text:
                    paragrafo.text = paragrafo.text.replace(chave, chave_de_substituicao[chave])
                      
        try:
            documento.save(f"Documentos Prontos\{peca} - {self.dicionario['CLIENTE_NOME']} x {self.dicionario['ADVERSO_NOME']}.docx")
        except:
            documento.save(f"Documentos Prontos\{peca} - {self.dicionario['CLIENTE_NOME']}.docx")

In [88]:
lista = ['Fabiano Darc Arantes', 'Sandra Segarra', 'R. N. Locação e Administração de Bens Próprios LTDA.']
cliente = 'Residencial Miosotis'

x = PECA('Procuração', cliente)
x = PECA('Procuração', lista)

print(type(lista))

<class 'list'>
